In [7]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install flickrapi


In [21]:
from flickrapi import FlickrAPI
import pandas as pd
import sys
key='4d860f720718fdce373fe644da0d61b4'
secret='813a60f10dffcbe6'


In [34]:
def get_urls(image_tag, MAX_COUNT):
    

    flickr = FlickrAPI(key, secret)
    photos = flickr.walk(text=image_tag,
                            tag_mode='all',
                            tags=image_tag,
                            has_geo=1,
                            lat=41.604168,
                            lon=-71.320683,
                            extras='date_upload, date_taken, geo, url_o',
                            per_page=50,
                            sort='relevance'
                        ) 
    count=0
    urls=[]
    uploads=[]
    geos=[]
    takens=[]
    for photo in photos:
        if count< MAX_COUNT:
            count=count+1
            print("Fetching url for image number {}".format(count))
            try:
                url=photo.get('url_o')
                urls.append(url)
                upload=photo.get('date_upload')
                uploads.append(upload)
                taken=photo.get('date_taken')
                takens.append(taken)
                geo=photo.get('geo')
                geos.append(geo)
            except:
                print("Url for image number {} could not be fetched".format(count))
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls),MAX_COUNT))
            break
    urls=pd.Series(urls)
    uploads=pd.Series(uploads)
    geos=pd.Series(geos)
    takens=pd.Series(takens)
    print("Writing out the urls in the current directory")
    urls.to_csv(image_tag+"_urls.csv")
    uploads.to_csv(image_tag+"_uploads.csv")
    takens.to_csv(image_tag+"_takens.csv")
    geos.to_csv(image_tag+"_geos.csv")
    print("Done!!!")



In [35]:
get_urls("fishing", 100)

Fetching url for image number 1
Fetching url for image number 2
Fetching url for image number 3
Fetching url for image number 4
Fetching url for image number 5
Fetching url for image number 6
Writing out the urls in the current directory
Done!!!


In [36]:
## Only need the following code if running this from Terminal
#def main(): 
#    tag=sys.argv[1]
#    MAX_COUNT=int(sys.argv[2])
#    get_urls(tag,MAX_COUNT)


#if __name__=='__main__': 
#    main() 
# if running the module as the main program (e.g., in terminal: python getFishDat.py): execute function (main)
# if module is imported by another program, _name_ is assigned to the module name "getFishDat": skip function (main)

import csv
import requests
import os
import sys
import time
def put_images(FILE_NAME):
    urls=[]
    with open(FILE_NAME,newline="") as csvfile:
        doc=csv.reader(csvfile,delimiter=",")
        for row in doc:
            if row[1].startswith("https"):
                urls.append(row[1])
    if not os.path.isdir(os.path.join(os.getcwd(),FILE_NAME.split("_")[0])):
        os.mkdir(FILE_NAME.split("_")[0])
    t0=time.time()
    for url in enumerate(urls):
        print("Starting download {} of ".format(url[0]+1),len(urls))
        try:
            resp=requests.get(url[1],stream=True)
            path_to_write=os.path.join(os.getcwd(),FILE_NAME.split("_")[0],url[1].split("/")[-1])
            outfile=open(path_to_write,'wb')
            outfile.write(resp.content)
            outfile.close()
            print("Done downloading {} of {}".format(url[0]+1,len(urls)))
        except:
            print("Failed to download url number {}".format(url[0]))
    t1=time.time()
    print("Done with download, job took {} seconds".format(t1-t0))





In [37]:
put_images("fishing_urls.csv")

Starting download 1 of  3
Done downloading 1 of 3
Starting download 2 of  3
Done downloading 2 of 3
Starting download 3 of  3
Done downloading 3 of 3
Done with download, job took 0.8717751502990723 seconds
